In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [2]:
def data():
    # Split train & validation set
    DDD_Train_Decile7 = pd.read_pickle("DDD_Train_Decile7.pkl")
    DDD_Train_Targets_1D = pd.read_pickle("DDD_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("DDD_Test_Decile7.pkl")
    y_test = pd.read_pickle("DDD_Test_Targets_1D.pkl")
    
    X_train, X_val, y_train, y_val = train_test_split(DDD_Train_Decile7, DDD_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones',
                moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None,
                beta_constraint=None, gamma_constraint=None))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,75,100,125,150])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM DDD Decile 7')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

2264/2264 [==============================] - 10s 5ms/step - loss: 0.3994 - acc: 0.8366 - val_loss: 0.3522 - val_acc: 0.8607
Epoch 36/75
2264/2264 [==============================] - 10s 5ms/step - loss: 0.4097 - acc: 0.8322 - val_loss: 0.3519 - val_acc: 0.8624
Epoch 37/75
2264/2264 [==============================] - 10s 4ms/step - loss: 0.3980 - acc: 0.8375 - val_loss: 0.3500 - val_acc: 0.8660
Epoch 38/75
2264/2264 [==============================] - 10s 5ms/step - loss: 0.3977 - acc: 0.8370 - val_loss: 0.3483 - val_acc: 0.8660
Epoch 39/75
2264/2264 [==============================] - 10s 4ms/step - loss: 0.3938 - acc: 0.8313 - val_loss: 0.3460 - val_acc: 0.8607
Epoch 40/75
2264/2264 [==============================] - 10s 4ms/step - loss: 0.4001 - acc: 0.8414 - val_loss: 0.3463 - val_acc: 0.8571
Epoch 41/75
2264/2264 [==============================] - 10s 4ms/step - loss: 0.3918 - acc: 0.8405 - val_loss: 0.3450 - val_acc: 0.8589
Epoch 42/75
2264/2264 [==============================] - 10s

Epoch 19/75
2264/2264 [==============================] - 10s 4ms/step - loss: 0.3438 - acc: 0.8467 - val_loss: 0.3059 - val_acc: 0.8589
Epoch 20/75
2264/2264 [==============================] - 10s 5ms/step - loss: 0.3433 - acc: 0.8538 - val_loss: 0.3057 - val_acc: 0.8589
Epoch 21/75
2264/2264 [==============================] - 11s 5ms/step - loss: 0.3382 - acc: 0.8547 - val_loss: 0.3061 - val_acc: 0.8589
Epoch 22/75
2264/2264 [==============================] - 10s 4ms/step - loss: 0.3459 - acc: 0.8463 - val_loss: 0.3066 - val_acc: 0.8589
Epoch 23/75
2264/2264 [==============================] - 11s 5ms/step - loss: 0.3424 - acc: 0.8516 - val_loss: 0.3059 - val_acc: 0.8589
Epoch 24/75
2264/2264 [==============================] - 10s 4ms/step - loss: 0.3437 - acc: 0.8516 - val_loss: 0.3053 - val_acc: 0.8589

Epoch 00024: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
Epoch 25/75
2264/2264 [==============================] - 10s 5ms/step - loss: 0.3393 - acc: 0.8556 - v

2264/2264 [==============================] - 11s 5ms/step - loss: 0.5668 - acc: 0.7496 - val_loss: 0.5136 - val_acc: 0.7672
Epoch 3/100
2264/2264 [==============================] - 10s 4ms/step - loss: 0.4917 - acc: 0.7937 - val_loss: 0.4216 - val_acc: 0.8307
Epoch 4/100
2264/2264 [==============================] - 10s 4ms/step - loss: 0.4387 - acc: 0.8171 - val_loss: 0.3819 - val_acc: 0.8536
Epoch 5/100
2264/2264 [==============================] - 10s 5ms/step - loss: 0.4223 - acc: 0.8140 - val_loss: 0.3699 - val_acc: 0.8571
Epoch 6/100
2264/2264 [==============================] - 10s 4ms/step - loss: 0.4102 - acc: 0.8326 - val_loss: 0.3553 - val_acc: 0.8554
Epoch 7/100
2264/2264 [==============================] - 10s 4ms/step - loss: 0.3984 - acc: 0.8370 - val_loss: 0.3443 - val_acc: 0.8554
Epoch 8/100
2264/2264 [==============================] - 10s 5ms/step - loss: 0.3851 - acc: 0.8379 - val_loss: 0.3268 - val_acc: 0.8589
Epoch 9/100
2264/2264 [==============================] - 10s

Epoch 61/100
2264/2264 [==============================] - 10s 4ms/step - loss: 0.3243 - acc: 0.8564 - val_loss: 0.2906 - val_acc: 0.8589
Epoch 62/100
2264/2264 [==============================] - 10s 4ms/step - loss: 0.3307 - acc: 0.8529 - val_loss: 0.2908 - val_acc: 0.8607
Epoch 63/100
2264/2264 [==============================] - 10s 5ms/step - loss: 0.3229 - acc: 0.8498 - val_loss: 0.2906 - val_acc: 0.8589
Epoch 64/100
2264/2264 [==============================] - 10s 4ms/step - loss: 0.3273 - acc: 0.8609 - val_loss: 0.2903 - val_acc: 0.8589
Epoch 65/100
2264/2264 [==============================] - 9s 4ms/step - loss: 0.3238 - acc: 0.8507 - val_loss: 0.2904 - val_acc: 0.8589
Epoch 66/100
2264/2264 [==============================] - 10s 5ms/step - loss: 0.3286 - acc: 0.8525 - val_loss: 0.2905 - val_acc: 0.8607
Epoch 67/100
2264/2264 [==============================] - 10s 4ms/step - loss: 0.3230 - acc: 0.8489 - val_loss: 0.2906 - val_acc: 0.8607
Epoch 68/100
2264/2264 [==================

...

2264/2264 [==============================] - 10s 4ms/step - loss: 0.4322 - acc: 0.8136 - val_loss: 0.3824 - val_acc: 0.8413
Epoch 24/125
2264/2264 [==============================] - 10s 4ms/step - loss: 0.4316 - acc: 0.8087 - val_loss: 0.3808 - val_acc: 0.8413
Epoch 25/125
2264/2264 [==============================] - 9s 4ms/step - loss: 0.4234 - acc: 0.8145 - val_loss: 0.3784 - val_acc: 0.8448
Epoch 26/125
2264/2264 [==============================] - 10s 5ms/step - loss: 0.4225 - acc: 0.8251 - val_loss: 0.3766 - val_acc: 0.8430
Epoch 27/125
2264/2264 [==============================] - 10s 4ms/step - loss: 0.4203 - acc: 0.8114 - val_loss: 0.3741 - val_acc: 0.8448
Epoch 28/125
2264/2264 [==============================] - 10s 4ms/step - loss: 0.4239 - acc: 0.8282 - val_loss: 0.3731 - val_acc: 0.8448
Epoch 29/125
2264/2264 [==============================] - 10s 4ms/step - loss: 0.4176 - acc: 0.8255 - val_loss: 0.3713 - val_acc: 0.8448
Epoch 30/125
2264/2264 [==============================]

Epoch 83/125
2264/2264 [==============================] - 10s 4ms/step - loss: 0.3570 - acc: 0.8454 - val_loss: 0.3133 - val_acc: 0.8642
Epoch 84/125
2264/2264 [==============================] - 9s 4ms/step - loss: 0.3674 - acc: 0.8520 - val_loss: 0.3151 - val_acc: 0.8624
Epoch 85/125
2264/2264 [==============================] - 10s 4ms/step - loss: 0.3614 - acc: 0.8511 - val_loss: 0.3149 - val_acc: 0.8624
Epoch 86/125
2264/2264 [==============================] - 9s 4ms/step - loss: 0.3607 - acc: 0.8476 - val_loss: 0.3143 - val_acc: 0.8589
Epoch 87/125
2264/2264 [==============================] - 10s 4ms/step - loss: 0.3584 - acc: 0.8507 - val_loss: 0.3132 - val_acc: 0.8571

Epoch 00087: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 88/125
2264/2264 [==============================] - 10s 4ms/step - loss: 0.3553 - acc: 0.8534 - val_loss: 0.3129 - val_acc: 0.8607
Epoch 89/125
2264/2264 [==============================] - 10s 5ms/step - loss: 0.3596 - acc: 0.8507 - val_loss: 0.3

Epoch 17/100
2264/2264 [==============================] - 10s 5ms/step - loss: 0.3411 - acc: 0.8551 - val_loss: 0.2991 - val_acc: 0.8589
Epoch 18/100
2264/2264 [==============================] - 10s 4ms/step - loss: 0.3507 - acc: 0.8507 - val_loss: 0.3018 - val_acc: 0.8571
Epoch 19/100
2264/2264 [==============================] - 10s 4ms/step - loss: 0.3428 - acc: 0.8481 - val_loss: 0.2989 - val_acc: 0.8571
Epoch 20/100
2264/2264 [==============================] - 10s 4ms/step - loss: 0.3391 - acc: 0.8494 - val_loss: 0.2944 - val_acc: 0.8660
Epoch 21/100
2264/2264 [==============================] - 10s 4ms/step - loss: 0.3414 - acc: 0.8507 - val_loss: 0.2950 - val_acc: 0.8607
Epoch 22/100
2264/2264 [==============================] - 7s 3ms/step - loss: 0.3430 - acc: 0.8516 - val_loss: 0.2940 - val_acc: 0.8624
Epoch 23/100
2264/2264 [==============================] - 9s 4ms/step - loss: 0.3363 - acc: 0.8569 - val_loss: 0.2941 - val_acc: 0.8695
Epoch 24/100
2264/2264 [===================

2264/2264 [==============================] - 10s 4ms/step - loss: 0.3285 - acc: 0.8516 - val_loss: 0.2943 - val_acc: 0.8624
Epoch 76/100
2264/2264 [==============================] - 10s 4ms/step - loss: 0.3293 - acc: 0.8569 - val_loss: 0.2944 - val_acc: 0.8624
Epoch 77/100
2264/2264 [==============================] - 10s 4ms/step - loss: 0.3276 - acc: 0.8516 - val_loss: 0.2942 - val_acc: 0.8624
Epoch 78/100
2264/2264 [==============================] - 10s 4ms/step - loss: 0.3270 - acc: 0.8503 - val_loss: 0.2942 - val_acc: 0.8624
Epoch 79/100
2264/2264 [==============================] - 10s 4ms/step - loss: 0.3276 - acc: 0.8489 - val_loss: 0.2940 - val_acc: 0.8607
Epoch 80/100
2264/2264 [==============================] - 10s 4ms/step - loss: 0.3298 - acc: 0.8564 - val_loss: 0.2941 - val_acc: 0.8607
Epoch 81/100
2264/2264 [==============================] - 9s 4ms/step - loss: 0.3320 - acc: 0.8520 - val_loss: 0.2942 - val_acc: 0.8607
Epoch 82/100
2264/2264 [==============================]

2264/2264 [==============================] - 9s 4ms/step - loss: 0.3416 - acc: 0.8520 - val_loss: 0.3037 - val_acc: 0.8642
Epoch 34/50
2264/2264 [==============================] - 10s 4ms/step - loss: 0.3467 - acc: 0.8560 - val_loss: 0.3038 - val_acc: 0.8642
Epoch 35/50
2264/2264 [==============================] - 10s 5ms/step - loss: 0.3405 - acc: 0.8525 - val_loss: 0.3035 - val_acc: 0.8642
Epoch 36/50
2264/2264 [==============================] - 10s 4ms/step - loss: 0.3379 - acc: 0.8511 - val_loss: 0.3034 - val_acc: 0.8624
Epoch 37/50
2264/2264 [==============================] - 10s 4ms/step - loss: 0.3336 - acc: 0.8569 - val_loss: 0.3033 - val_acc: 0.8642
Epoch 38/50
2264/2264 [==============================] - 10s 4ms/step - loss: 0.3365 - acc: 0.8538 - val_loss: 0.3032 - val_acc: 0.8642
Epoch 39/50
2264/2264 [==============================] - 10s 4ms/step - loss: 0.3431 - acc: 0.8489 - val_loss: 0.3033 - val_acc: 0.8642
Epoch 40/50
2264/2264 [==============================] - 10s 

In [ ]:
0.86434